In [6]:
import pandas as pd
import numpy as np
import calendar

# get all tickers as list (tickers from _data/tickers.txt)
# tickers = "A AAL AAPL ABBV ABT ACGL ACN ADBE ADI ADM ADP ADSK AEE AEP AES AFL AIG AIZ AJG AKAM ALB ALGN ALK ALL ALLE AMAT AMCR AMD AME AMGN AMP AMT AMZN ANET ANSS AON AOS APA APD APH APTV ARE ATO ATVI AVB AVGO AVY AWK AXON AXP AZO BA BAC BALL BAX BBWI BBY BDX BEN BF.B BG BIIB BIO BK BKNG BKR BLK BMY BR BRK.B BRO BSX BWA BXP C CAG CAH CARR CAT CB CBOE CBRE CCI CCL CDAY CDNS CDW CE CEG CF CFG CHD CHRW CHTR CI CINF CL CLX CMA CMCSA CME CMG CMI CMS CNC COF COO COP COR COST CPB CPRT CPT CRL CRM CSCO CSGP CSX CTAS CTLT CTRA CTSH CTVA CVS CVX CZR D DAL DD DE DFS DG DGX DHI DHR DIS DLR DLTR DOV DOW DPZ DRI DTE DUK DVA DVN DXC DXCM EA EBAY ECL ED EFX EG EIX EL ELV EMN EMR ENPH EOG EPAM EQIX EQR EQT ES ESS ETN ETR ETSY EVRG EW EXC EXPD EXPE EXR F FANG FAST FCX FDS FDX FE FFIV FI FICO FIS FITB FLT FMC FOX FOXA FRT FSLR FTNT FTV GD GE GEHC GEN GILD GIS GL GLW GM GNRC GOOG GOOGL GPC GPN GRMN GS GWW HAL HAS HBAN HCA HD HES HIG HII HLT HOLX HON HPE HPQ HRL HSIC HST HSY HUM HWM IBM ICE IDXX IEX IFF ILMN INCY INTC INTU INVH IP IPG IQV IR IRM ISRG IT ITW IVZ J JBHT JCI JKHY JNJ JNPR JPM K KDP KEY KEYS KHC KIM KLAC KMB KMI KMX KO KR KVUE L LDOS LEN LH LHX LIN LKQ LLY LMT LNC LNT LOW LRCX LUV LVS LW LYB LYV MA MAA MAR MAS MCD MCHP MCK MCO MDLZ MDT MET META MGM MHK MKC MKTX MLM MMC MMM MNST MO MOH MOS MPC MPWR MRK MRNA MRO MS MSCI MSFT MSI MTB MTCH MTD MU NCLH NDAQ NDSN NEE NEM NFLX NI NKE NOC NOV NOW NRG NSC NTAP NTRS NUE NVDA NVR NWL NWS NWSA O ODFL OGN OKE OMC ON ORCL ORLY OTIS OXY PANW PARA PAYC PAYX PCAR PCG PEAK PEG PEP PFE PG PGR PH PHM PKG PLD PM PNC PNR PNW PODD POOL PPG PPL PRU PSA PSX PTC PWR PXD PYPL QCOM QRVO RCL REG REGN RF RHI RJF RL RMD ROK ROL ROP ROST RSG RTX RVTY SBAC SBUX SCHW SHEDG SEE SHW SJM SLB SNA SNPS SO SPG SPGI SRE STE STLD STT STX STZ SWK SWKS SYF SYK SYY T TAP TDG TDY TECH TEL TER TFC TFX TGT TJX TMO TMUS TPR TRGP TRMB TROW TRV TSCO TSLA TSN TT TTWO TXN TXT TYL UAL UDR UHS ULTA UNH UNP UPS URI USB V VFC VICI VLO VMC VRSK VRSN VRTX VTR VTRS VZ WAB WAT WBA WBD WDC WEC WELL WFC WHR WM WMB WMT WRB WRK WST WTW WY WYNN XEL XOM XRAY XYL YUM ZBH ZBRA ZION ZTS AAPL MSFT AMZN NVDA GOOGL GOOG TSLA AVGO ADBE COST PEP CSCO NFLX CMCSA AMD TMUS INTC INTU TXN AMGN AMAT HON QCOM BKNG SBUX ISRG ADP MDLZ GILD LRCX REGN ADI VRTX MU PANW MELI SNPS KLAC PYPL CDNS CHTR MAR CSX ABNB MNST ORLY PDD ASML NXPI WDAY CTAS FTNT LULU MRVL ADSK KDP ODFL PCAR PAYX MCHP CPRT ON MRNA ROST KHC DXCM AZN EXC AEP IDXX SGEN CRWD BIIB BKR TTD CTSH VRSK CEG CSGP EA GFS XEL TEAM FAST GEHC DDOG FANG WBD ANSS DLTR ALGN ILMN ZS EBAY WBA ZM SIRI ENPH JD LCID SPY QQQ IWM DIA"
# tickers = "AAPL MSFT AMZN GOOG NVDA TSLA META"  # shorter list for testing
# tickers = tickers.split(" ")

# get list of all file names in ../_data/day/bars
import os
day_bar_files = os.listdir("../_data/day/bars")
tickers = [x.replace(".csv", "") for x in day_bar_files]

# load day bar data from ../_data/day/bars as a dict of dataframes ( key: ticker, value: dataframe of day bars )
day_bars = {}
for ticker in tickers:
    day_bars[ticker] = pd.read_csv(f"../_data/day/bars/{ticker}.csv", index_col=None, parse_dates=True)

# preview day bar data for AAPL
day_bars["AAPL"].head()

,date,open,high,low,close,volume
0,2015-12-01,118.75,118.81,116.86,117.34,34852374
1,2015-12-02,117.05,118.11,116.08,116.28,33385643
2,2015-12-03,116.55,116.79,114.22,115.20,41560785
3,2015-12-04,115.29,119.25,115.11,119.03,57776977
4,2015-12-07,118.98,119.86,117.81,118.28,32080754


In [7]:
# for each ticker, add gap column to dataframe
# gap = if open > high of previous day, then gap = open - high of previous day (as percent change from previous day close)
#       else if open < low of previous day, then gap = open - low of previous day (as percent change from previous day close)
for ticker in tickers:
    df = day_bars[ticker]
    df["gap"] = 0.0  # Initialize the gap column with 0.0

    # Shift the 'high' and 'low' columns to get the previous day's values
    df["prev_high"] = df["high"].shift(1)
    df["prev_low"] = df["low"].shift(1)
    df["prev_close"] = df["close"].shift(1)

    # Calculate the gap based on the conditions you provided
    df["gap"] = np.where(
        df["open"] > df["prev_high"],
        (df["open"] - df["prev_high"]) / df["prev_close"],
        np.where(
            df["open"] < df["prev_low"],
            (df["open"] - df["prev_low"]) / df["prev_close"],
            0.00,
        ),
    )

    # Drop the temporary columns
    df.drop(["prev_high", "prev_low", "prev_close"], axis=1, inplace=True)


# preview day bar data for AAPL where gap is not 0
day_bars["AAPL"][day_bars["AAPL"]["gap"] != 0.0].head()

,date,open,high,low,close,volume,gap
5,2015-12-08,117.52,118.60,116.86,118.23,34309450,-0.002452
8,2015-12-11,115.00,115.39,112.85,113.18,46886161,-0.004390
9,2015-12-14,112.18,112.68,109.79,112.48,64941651,-0.005920
12,2015-12-17,112.02,112.25,108.98,108.98,44772827,0.000269
13,2015-12-18,108.91,109.52,105.81,106.03,96168327,-0.000642


In [8]:
# create gap_filled columns
# gap_up_filled = true if gap > 0 && low <= previous day high else false
# gap_down_filled = true if gap < 0 && high >= previous day low else false
for ticker in tickers:
    df = day_bars[ticker]

    # Shift the 'high' and 'low' columns to get the previous day's values
    df["prev_high"] = df["high"].shift(1)
    df["prev_low"] = df["low"].shift(1)

    # Calculate if gaps closed
    df["gap_up_filled"] = np.where((df["gap"] > 0) & (df["low"] <= df["prev_high"]), True, False)
    df["gap_down_filled"] = np.where((df["gap"] < 0) & (df["high"] >= df["prev_low"]), True, False)

    df["gap_filled"] = np.where(df["gap_up_filled"] | df["gap_down_filled"], True, False)

    # Drop the temporary columns
    df.drop(["prev_high", "prev_low", "gap_up_filled", "gap_down_filled"], axis=1, inplace=True)

# preview day bar data for AAPL where gap_up_filled is true
day_bars["AAPL"][day_bars["AAPL"]["gap_filled"] == True].head()

,date,open,high,low,close,volume,gap,gap_filled
5,2015-12-08,117.52,118.60,116.86,118.23,34309450,-0.002452,True
12,2015-12-17,112.02,112.25,108.98,108.98,44772827,0.000269,True
13,2015-12-18,108.91,109.52,105.81,106.03,96168327,-0.000642,True
15,2015-12-22,107.40,107.72,106.45,107.23,32789367,0.000280,True
17,2015-12-24,109.00,109.00,107.95,108.03,13596680,0.001381,True


In [9]:
# calculate profitability of each filled gap
# store results in a new dataframe with columns: date, ticker, gap, profit_at_close
# profit_at_close = if gap > 0, then (previous day high - close) / close else (close - previous day low) / previous day low
trades = pd.DataFrame(columns=["date", "ticker", "gap", "profit_at_close"])
for ticker in tickers:
    df = day_bars[ticker]

    # Shift the 'high' and 'low' columns to get the previous day's values
    df["prev_high"] = df["high"].shift(1)
    df["prev_low"] = df["low"].shift(1)

    # Calculate the profit_at_close based on the conditions you provided
    df["profit_at_close"] = np.where(
        df["gap"] > 0,
        (df["prev_high"] - df["close"]) / df["close"],
        (df["close"] - df["prev_low"]) / df["prev_low"],
    )

    # Calculate max profit
    df["max_profit"] = np.where(
        df["gap"] > 0,
        (df["prev_high"] - df["low"]) / df["low"],
        (df["high"] - df["prev_low"]) / df["prev_low"],
    )

    # Drop the temporary columns
    df.drop(["prev_high", "prev_low"], axis=1, inplace=True)

    # Select rows where gap_filled is True and append to trades
    selected_rows = df[df["gap_filled"] == True]

    # Select specific columns from selected rows
    selected_rows = selected_rows[['date', 'gap', 'profit_at_close', 'max_profit']]

    # Add ticker column
    selected_rows['ticker'] = ticker

    # Append the results to the 'trades' DataFrame
    trades = pd.concat([trades, selected_rows])

# preview trades dataframe
trades.head()


,date,ticker,gap,profit_at_close,max_profit
5,2015-12-08,CSCO,-0.003274,-0.004035,0.001101
6,2015-12-09,CSCO,-0.002210,-0.010733,0.012213
13,2015-12-18,CSCO,-0.003743,-0.016105,0.004494
15,2015-12-22,CSCO,0.007508,-0.008925,0.003766
18,2015-12-28,CSCO,-0.002557,-0.002192,0.000000


In [10]:
# results for all gap trades

# open file to write to named: backtest_gap_day_strategy.txt
file = open("backtest_gap_day_strategy.txt", "w")

# calculate average profit_at_close over all trades
avg_profit_at_close = trades["profit_at_close"].mean()
avg_profit_at_close_gap_up = trades[trades["gap"] > 0]["profit_at_close"].mean()
avg_profit_at_close_gap_down = trades[trades["gap"] < 0]["profit_at_close"].mean()

# calculate average max_profit over all trades
avg_max_profit = trades["max_profit"].mean()
avg_max_profit_gap_up = trades[trades["gap"] > 0]["max_profit"].mean()
avg_max_profit_gap_down = trades[trades["gap"] < 0]["max_profit"].mean()

# calculate precentage of profitable trades
pct_profitable = len(trades[trades["profit_at_close"] > 0]) / len(trades)
pct_profitable_gap_up = len(trades[(trades["gap"] > 0) & (trades["profit_at_close"] > 0)]) / len(trades[trades["gap"] > 0])
pct_profitable_gap_down = len(trades[(trades["gap"] < 0) & (trades["profit_at_close"] > 0)]) / len(trades[trades["gap"] < 0])

# calculate average profit_at_close for each ticker
avg_profit_at_close_by_ticker = trades.groupby("ticker")["profit_at_close"].mean()
avg_profit_at_close_by_ticker_gap_up = trades[trades["gap"] > 0].groupby("ticker")["profit_at_close"].mean()
avg_profit_at_close_by_ticker_gap_down = trades[trades["gap"] < 0].groupby("ticker")["profit_at_close"].mean()

# calculate average profit_at_close for each year
trades['date'] = pd.to_datetime(trades['date'])
trades['year'] = trades['date'].dt.year
avg_profit_at_close_by_year = trades.groupby("year")["profit_at_close"].mean()
avg_profit_at_close_by_year_gap_up = trades[trades["gap"] > 0].groupby("year")["profit_at_close"].mean()
avg_profit_at_close_by_year_gap_down = trades[trades["gap"] < 0].groupby("year")["profit_at_close"].mean()

# calculate average profit_at_close for each month
trades['month'] = trades['date'].dt.month
avg_profit_at_close_by_month = trades.groupby("month")["profit_at_close"].mean()
avg_profit_at_close_by_month_gap_up = trades[trades["gap"] > 0].groupby("month")["profit_at_close"].mean()
avg_profit_at_close_by_month_gap_down = trades[trades["gap"] < 0].groupby("month")["profit_at_close"].mean()

# write results to file

file.write(f"--------------------------------------------------\n")
file.write(f"Gap-up Results\n")
file.write(f"--------------------------------------------------\n\n")

file.write(f"Percentage profitable   = {(pct_profitable_gap_up * 100):05.2f}%\n")
file.write(f"Average profit at close = {(avg_profit_at_close_gap_up * 100):05.2f}%\n")
file.write(f"Average max profit      = {(avg_max_profit_gap_up * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by year:\n")
# for year in avg_profit_at_close_by_year_gap_up.index:
#     file.write(f"  {year:4d}: {(avg_profit_at_close_by_year_gap_up[year] * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by month:\n")
# for month in avg_profit_at_close_by_month_gap_up.index:
#     file.write(f"  {calendar.month_name[month]:9s}: {(avg_profit_at_close_by_month_gap_up[month] * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by ticker:\n")
# for ticker in tickers:
#     file.write(f"  {ticker:4s}: {(avg_profit_at_close_by_ticker_gap_up[ticker] * 100):05.2f}%\n")

file.write(f"\n\n")

file.write(f"--------------------------------------------------\n")
file.write(f"Gap-down Results\n")
file.write(f"--------------------------------------------------\n\n")

file.write(f"Percentage profitable   = {(pct_profitable_gap_down * 100):05.2f}%\n")
file.write(f"Average profit at close = {(avg_profit_at_close_gap_down * 100):05.2f}%\n")
file.write(f"Average max profit      = {(avg_max_profit_gap_down * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by year:\n")
# for year in avg_profit_at_close_by_year_gap_down.index:
#     file.write(f"  {year:4d}: {(avg_profit_at_close_by_year_gap_down[year] * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by month:\n")
# for month in avg_profit_at_close_by_month_gap_down.index:
#     file.write(f"  {calendar.month_name[month]:9s}: {(avg_profit_at_close_by_month_gap_down[month] * 100):05.2f}%\n")

# file.write(f"\nAverage profit at close by ticker:\n")
# for ticker in tickers:
#     file.write(f"  {ticker:4s}: {(avg_profit_at_close_by_ticker_gap_down[ticker] * 100):05.2f}%\n")

file.close()
